# Minimum Package Requirement

We have used PyPI to host our package. Users can download our Automatic Differentiation package with the following command:

`pip install autodiffing`

To install the required dependencies, users need to run the following command:

`pip install -r requirements.txt`

Within our requirements.txt, we have the following packages:

`bleach==3.1.0`\
`certifi==2019.9.11`\
`cycler==0.10.0`\
`docutils==0.15.2`\
`kiwisolver==1.1.0`\
`matplotlib==3.1.1`\
`nltk==3.4.5`\
`numpy==1.17.4`\
`Pygments==2.4.2`\
`pyparsing==2.4.5`\
`python-dateutil==2.8.1`\
`readme-renderer==24.0`\
`requests-toolbelt==0.9.1`\
`scipy==1.3.2`\
`six==1.13.0`\
`twine==2.0.0`\
`webencodings==0.5.1`\

Most of the packages above come with the installation of `python` version 3.7. 

Our team has selected only 3 additional packages for our user to install, namely `matplotlib`, `scipy` and `numpy`. 

`numpy` is essential for our Automatic Differentiation package as we require it for the calculation of our elementary functions, and for dealing with arrays and matrices when there are vector functions and vector inputs.

`scipy` is a good package to have for its optimization and linear algebra abilities. In particular, under our future features for the Automatic Differentiation package, we hope to be able to deal with optimization problems. 

`matplotlib` is needed for any potential visualization of our outputs. `bleach`, `docutils`, `Pygments` are additional packages that `matplotlib` requires.



# Future Features

Our future features for our Automatic Differentiation package include taking in vector inputs and vector functions and implementing reverse mode for automatic differentiation.

In order for our user to 

Now that you've got most of the hard implementation work done, what kinds of things do you want to impelement next? How will your software change? What will be the primary challenges to implementing these new features? Things you may want to consider here include (but are not limited to) any changes to the directory structure, and new modules, classes, data structures, etc.

We are checking for reasonable ideas here. If you really can't come up with anything interesting, you should consult the lecture notes. If you are still lost after that, please ask your TF or the instructor for help. If you are unsure about the viability or difficulty of your proposed extension, please ask your TF or the instructor for their thoughts. The extension should not be trivial and you should have given it sufficient thought that you can write something concrete about how to proceed. Please be creative and have fun with it!

In [1]:
# Libraries

import numpy as np

In [30]:
class DualNumber():
    '''
    Description: a class to hold dual number representations of vectors/scalars.
    '''

    def __init__(self, real, dual=1):
        self.input_type_check(real)
        self.input_type_check(dual)
        self._val = real
        self._der = dual
    
    def val(self):
        return self._val
    
    def der(self):
        return self._der
    
# Overloading arithmetic operators

    def __add__(self, other):
        try:
            val2 = self._val + other._val
            der2 = self._der + other._der
            return DualNumber(val2, der2)
        except AttributeError:
            self.real_check(other)
            val2 = self._val + other
            der2 = self._der
            return DualNumber(val2, der2)

    def __radd__(self, other):
        return self.__add__(other)

    def __mul__(self, other):
        try:
            val2 = self._val * other._val
            der2 = self._der * other._val + self._val * other._der
            return DualNumber(val2, der2)
        except AttributeError:
            self.real_check(other)
            val2 = self._val * other
            der2 = self._der * other
            return DualNumber(val2, der2)

    def __rmul__(self, other):
        return self.__mul__(other)

    def __sub__(self, other):
        try:
            val2 = self._val - other._val
            der2 = self._der - other._der
            return DualNumber(val2, der2)
        except AttributeError:
            self.real_check(other)
            val2 = self._val - other
            der2 = self._der
            return DualNumber(val2, der2)

    def __rsub__(self, other):
        try:
            val2 = other._val - self._val
            der2 = other._der - self._der
            return DualNumber(val2, der2)
        except AttributeError:
            self.real_check(other)
            val2 = other - self._val
            der2 = -self._der
            return DualNumber(val2, der2)

    def __truediv__(self, other):
        try:
            val2 = self._val / other._val
            der2 = (self._der * other._val - self._val*other._der)/(self._val*self._val)
            return DualNumber(val2, der2)
        except AttributeError:
            self.real_check(other)
            val2 = self._val / other
            der2 = self._der / other
            return DualNumber(val2, der2)

    def __rtruediv__(self, other):
        try:
            val2 = other._val / self._val
            der2 = (other._der * self._val - other._val*self._der)/(other._val*other._val)
            return DualNumber(val2, der2)
        except AttributeError:
            self.real_check(other)
            val2 = other / self._val
            der2 = -other*self._der / (self._val*self._val)
            return DualNumber(val2, der2)

    def __pow__(self, other):
        try:
            val2 = self._val ** other._val
            der2 = val2*(other._val/self._val*self._der+other._der*np.log(self._val))
            return DualNumber(val2, der2)
        except AttributeError:
            self.real_check(other)
            val2 = self._val ** other
            der2 = other * (self._val ** (other - 1)) * self._der
            return DualNumber(val2, der2)


    def __rpow__(self, other):
        try:
            val2 = other._val ** self._val
            der2 = val2*(self._val/other._val*other._der+self._der*np.log(other._val))
            return DualNumber(val2, der2)
        except AttributeError:
            self.real_check(other)
            val2 = other ** self._val
            der2 = other ** self._val * np.log(other)
            return DualNumber(val2, der2)

# Overloading unary operators
    def __pos__(self):
        val2 = self._val
        der2 = self._der
        return DualNumber(val2, der2)

    def __neg__(self):
        val2 = -self._val
        der2 = -self._der
        return DualNumber(val2, der2)

    def __abs__(self):
        val2 = abs(self._val)
        der2 = abs(self._der)
        return DualNumber(val2, der2)

    def __round__(self, n=None):
        val2 = round(self._val, n)
        der2 = round(self._der, n)
        return DualNumber(val2, der2)
    
# Other dunder methods
    def __repr__(self):
        return 'DualNumber({},{})'.format(self._val,self._der)

# Check Data Types
    @staticmethod        
    def input_type_check(x):
        assert (isinstance(x, float) or isinstance(x, int)) or isinstance(x, DualNumber), 'Check the data type of {}!'.format(x)

    @staticmethod        
    def real_check(x):
        assert(isinstance(x, float) or isinstance(x, int)), 'Check the data type of {}!'.format(x)

if __name__ =="__main__":
    x=DualNumber(-2.578,1)
    y=DualNumber(3,1)
    f=x**2
    print(f.val(),f.der())

6.646083999999999 -5.156


In [14]:
class Sin(DualNumber):
    def __init__(self, x):
        self._val = np.sin(x._val)
        self._der = np.cos(x._val)*x._der
 
    
class Tan(DualNumber):
    def __init__(self, x):
        self._val = np.tan(x._val)
        self._der = (1+np.tan(x._val)*np.tan(x._val))*x._der
        

class Cos(DualNumber):
    def __init__(self, x):
        self._val = np.cos(x._val)
        self._der = -1*np.sin(x._val)*x._der


class Exp(DualNumber):
    def __init__(self, x):
        self._val = np.exp(x._val)
        self._der = np.exp(x._val)*x._der


class Power(DualNumber):
    def __init__(self, x, n):
        self._val = x._val**n
        self._der = n*(x._val**(n-1))*x._der
        

class Log(DualNumber):
    def __init__(self, x):
        self._val = np.log(x._val)
        self._der = (1/x._val)*x._der
        
        
class ArcSin(DualNumber):
    def __init__(self, x):
        self._val = np.arcsin(x._val)
        try:
            self._der = 1/np.sqrt(1-x._val**2) * x._der
        except Exception as e:
            print(f'ArcSin has domain (-1,1)!{e}')


class ArcCos(DualNumber):
    def __init__(self, x):
        self._val = np.arccos(x._val)
        assert abs(x.val) <= 1
        self._der = -1/np.sqrt(1-x._val**2) * x._der

            
            
class ArcTan(DualNumber):
    def __init__(self, x):
        self._val = np.arctan(x._val)
        self._der = 1/np.sqrt(1+x._val**2) * x._der


class Sqrt(DualNumber):
    def __init__(self, x):
        self._val = np.sqrt(x._val)
        self._der = 1/(2*np.sqrt(x._val)) * x._der



In [ ]:
x = Cos()

In [ ]:
# check assert type